# Import Necessary Packages

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5,6,7"

import torch
import torch.nn as nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image 

from tqdm import tqdm

# Pytorch Version
print(torch.__version__)
print(torch.cuda.is_available())

1.7.0
True


# Hyper Parameters

In [2]:
batch_size = 128
num_epoch = 200
z_dimension = 100 # size of input noise
lr_G = 1e-4
lr_D = 1e-4 # leanring rate of Generator and Discriminator


# Datasets files & utils

In [3]:
# the path we save data
if not os.path.exists('../img'):
    os.mkdir('../img')

# Transform of our datasets
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean = (0.5), std = (0.5))
])
    
# MNIST datasets downloading……
mnist = datasets.MNIST(
    root = '../data/mnist/', train = True, transform = trans, download = True
)

# Dataloader
dataloader = torch.utils.data.DataLoader(
    dataset = mnist, batch_size = batch_size, shuffle = True
)


In [4]:
def to_gray_img(x):
    out = 0.5*(x+1)
    out = out.clamp(0,1) # force therandom distribution of x into [min, max]
    out = out.view(-1, 1, 28, 28) # view a multi-row Tensor as one row
    return out

# Discriminator Definitions (Vanilla FC Fashion)
Considering that the MNIST images are all of 28 * 28 images, the input size of our fully connection network is 784.

It is a **unconditional** version.

In [5]:
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__() # 说起来这句是干嘛的？
        self.D = nn.Sequential(
            nn.Linear(784, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid() # propability of the given image being Real
        )
    
    def forward(self, x):
        x = self.D(x)
        return x

# Generator Definitions (Vanilla FC Fashion)
Considering that the MNIST images are all of 28 * 28 images, the output size of our fully connection network is 784.

The final `Tanh` activataion function can project output into (-1,1)

It is a **unconditional** version.

In [6]:
class generator(nn.Module):
    
    def __init__(self):
        super(generator, self).__init__()
        self.G = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(True),
            nn.Linear(256, 256),
            nn.ReLU(True),
            nn.Linear(256, 784),
            nn.Tanh()
        )
        
    def forward(self, x):
        x = self.G(x)
        return x
        
        

# Traning Phase
## Model & Opitmizer

In [16]:

D = discriminator()
G = generator()


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

D = D.to(device)
G = G.to(device)




In [17]:
criterion = nn.BCELoss() # 二分类交叉熵

d_optimizer = torch.optim.Adam(D.parameters(), lr = lr_D)
g_optimizer = torch.optim.Adam(G.parameters(), lr = lr_G)

## Trainning Start!

In [24]:
for epoch in tqdm(range(num_epoch)):
    for i, (img, _) in enumerate(dataloader): # Since we don't need any labels, we make label be '_'

        # =============================== IMAGES =================================
        num_img = img.size(0) 
        img = img.view(num_img, -1) # view()函数作用是将一个多行的Tensor,拼接成一行
                                    # 第一个参数是要拼接的tensor,第二个参数是-1
        # After view(), the input image will be a vector as 28 * 28 = 782 dimensions
        real_img = img.to(device) # the sampled data are real images
        # =============================== LABELS =================================
        real_label = torch.ones(num_img).view(-1, 1).to(device) # Label of real images is 1
        fake_label = torch.zeros(num_img).view(-1, 1).to(device) # label of real images is 0
        
        # ========================= D: Forward and Loss ==========================
        
        real_out = D(real_img)
        d_loss_real = criterion(real_out, real_label) # TODO: criterion
        
        z = torch.randn(num_img, z_dimension).to(device)

        fake_img = G(z)
        fake_out = D(fake_img)
        d_loss_fake = criterion(fake_out, fake_label)
        
        d_loss = d_loss_real + d_loss_fake
        d_optimizer.zero_grad() # before back propagation
        d_loss.backward()
        d_optimizer.step()
        
        # =============== G ========================
        # To fool the Discriminator, we want G's fake output can get 'real' pridection
        z = torch.randn(num_img, z_dimension).to(device)
        fake_img = G(z)
        fake_out = D(fake_img)
        g_loss = criterion(fake_out, real_label)
        
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
        # ============= Visualization ===============
        if (i+1)%100 == 0:
            print('Epoch[{}/{}],d_loss:{:.6f},g_loss:{:.6f} '
                  'D real: {:.6f},D fake: {:.6f}'.format(
                epoch,num_epoch,d_loss.item(),g_loss.item(),
                real_out.data.mean(),fake_out.data.mean()  #打印的是真实图片的损失均值
            ))
        
    if epoch == 0:
        real_images = to_gray_img(real_img.cpu().data)
        save_image(real_images, '../img/real_images.png')
 
    fake_images = to_gray_img(fake_img.cpu().data)
    save_image(fake_images, '../img/fake_images-{}.png'.format(epoch+1))

  0%|          | 0/100 [00:00<?, ?it/s]

Epoch[0/100],d_loss:0.452047,g_loss:2.703708 D real: 0.847078,D fake: 0.149786
Epoch[0/100],d_loss:0.527328,g_loss:2.918219 D real: 0.810040,D fake: 0.117311
Epoch[0/100],d_loss:0.608762,g_loss:3.089234 D real: 0.874103,D fake: 0.115298
Epoch[0/100],d_loss:0.539014,g_loss:2.693022 D real: 0.765134,D fake: 0.140833


  1%|          | 1/100 [00:43<1:11:52, 43.56s/it]

Epoch[1/100],d_loss:0.413162,g_loss:3.255656 D real: 0.869549,D fake: 0.108588
Epoch[1/100],d_loss:0.427933,g_loss:3.135616 D real: 0.817288,D fake: 0.083811
Epoch[1/100],d_loss:0.451787,g_loss:2.734869 D real: 0.834084,D fake: 0.126727
Epoch[1/100],d_loss:0.411406,g_loss:2.821395 D real: 0.826763,D fake: 0.131237


  2%|▏         | 2/100 [01:18<1:06:53, 40.95s/it]

...

 98%|█████████▊| 98/100 [53:35<01:03, 31.69s/it]

Epoch[98/100],d_loss:0.693967,g_loss:1.426769 D real: 0.805623,D fake: 0.321001
Epoch[98/100],d_loss:0.842177,g_loss:1.683418 D real: 0.707723,D fake: 0.273339
Epoch[98/100],d_loss:0.871908,g_loss:1.607235 D real: 0.718410,D fake: 0.265457
Epoch[98/100],d_loss:0.705924,g_loss:1.698898 D real: 0.751922,D fake: 0.273065


 99%|█████████▉| 99/100 [54:06<00:31, 31.55s/it]

Epoch[99/100],d_loss:0.856721,g_loss:1.601435 D real: 0.707706,D fake: 0.273405
Epoch[99/100],d_loss:1.000444,g_loss:1.507777 D real: 0.645780,D fake: 0.285567
Epoch[99/100],d_loss:0.740998,g_loss:1.669176 D real: 0.759928,D fake: 0.260263
Epoch[99/100],d_loss:0.860434,g_loss:1.456508 D real: 0.750482,D fake: 0.320452


100%|██████████| 100/100 [54:38<00:00, 32.79s/it]


# Debug 专区

In [19]:
print(real_out.size())

torch.Size([128, 1])


In [ ]:
torch.save(G.state_dict(),'./generator.pth')
torch.save(D.state_dict(),'./discriminator.pth')